In [78]:
from collections import defaultdict
import numpy as np
import pandas as pd

# Document Classification with Naive Bayes
Based on [Jurafsky: Speech and Language Processing (3rd ed. draft)](https://web.stanford.edu/~jurafsky/slp3/)

Multinomial Naive Bayes: The class priors are assumed to follow a multinomial distribution - common for text classification.

Assumes Bag-of-Words: Documents are represented by the unordered collection of their word counts. Text sequence information is ignored.
## The Model
* Vocabulary $V$: set of words $\{w_j\}$
* Feature based on vocabulary word $w_j$ in document $d$: $x_{n,j}=f(w_j;D_n)$  
* $K$ Classes $c_1,...,c_K$  
* $N$ Documents
The Naive Bayes classifier returns the class $\hat{c}$ which has the maximum posterior probability given the document $D$.
$$
\DeclareMathOperator*{\argmax}{argmax}
\hat{c}=\argmax_{c \in C} P(c|D)
$$

Bayes' Theorem says that the conditional probability $P(c|D)$ to observe class $c$ given document $D$ can be decomposed into the product of the likelihood $P(D|c)$ and the prior $P(c)$ (the probability for class $c$ without any observation)
$$\hat{c}=\argmax_{c \in C} P(c|D)=\argmax_{c \in C} \frac{P(D|c)P(c)}{P(D)}$$

$P(D)$ can be dropped, because it is the same for all classes.

$$\hat{c}=\argmax_{c \in C} P(c|D)=\argmax_{c \in C} \overbrace{P(D|c)}^{\text{likelihood}}\overbrace{P(c)}^{\text{prior}}$$

Represent document $D$ as a set of features $f_1, f_2,...,f_J$, which encode the presence of the respective vocabulary words $w_1, w_2,...,w_J$  

$$\hat{c}=\argmax_{c \in C} P(f_1, f_2,...,f_J|c)P(c)$$

Naive assumption: the probabilities $P(f_j|c)$ are independent given the class $c$ and therefore

$$P(f_1, f_2,...,f_J|c) = P(f_j|c)\cdot P(f_2|c)\cdot ... \cdot P(f_J|c)$$

$$c_{NB}=\argmax_{c \in C} P(c) \prod_{f} P(f|c)$$

in log space to avoid underflow and increase speed:

$$c_{NB}=\argmax_{c \in C} \log P(c) + \sum_{f} \log P(f|c)$$

Applied to word tokens of document $D$ $\{t_i\}$, where $w_i$ is the corresponding word in the vocabulary. Tokens $t_i$ without corresponding word in the vocabulary are simply ignored:
$$c_{NB}=\argmax_{c \in C} \log P(c) + \sum_{t_i} \log P(w_i|c)$$

## Training

$$\hat{P}(c)=\frac{N_c}{N}$$
$N_c$ the number of documents of class $c$ divided by the total number of documents/samples.

$P(w_{j}|c)$ is computed as the fraction of times the vocabulary word $w_j$ appears among all tokens in all documents of class $c$

$$P(w_{i}|c)=\frac{count(w_j,c)}{\sum_{w_{j'}\in V}count(w_{j'},c)}$$

Apply Smoothing $\alpha$ to avoid zero probabilites for vocabulary words that do not appear in all classes.

$$P(w_{i}|c)=\frac{count(w_i,c)+\alpha}{\sum_{w\in V}(count(w,c)+\alpha)}=\frac{count(w_i,c)+\alpha}{\left(\sum_{w\in V}(count(w,c)\right)+\alpha\cdot|V|}$$

Laplace Smoothing: $\alpha = 1$  
Lidstone Smoothing: $0 < \alpha < 1$ 

Words missing in the vocabulary are simply ignored.

## The Data
Sentiment Classification task (0: Negative, 1: Positive)

In [100]:
docs = ['just plain boring',
         'entirely predictable and lacks energy',
         'no surprises and very few laughs',
         'very powerful',
         'the most fun file of the summer'
        ]

classes = ['-',
     '-',
     '-',
     '+',
     '+'
    ]

In [163]:
N = len(docs)
clss = list(set(classes))
K = len(clss)

## An Illustrative Implementation

In [229]:
alpha = 1.0

### Organise the Target Classes

In [164]:
map_cls = {label:i for i,label in enumerate(clss)}

T = np.array([map_cls[label] for label in classes])

In [165]:
clss

['+', '-']

In [166]:
T

array([1, 1, 1, 0, 0])

### Calculate the Log Priors

In [167]:
log_priors = [0.0]*K
for t in T:
    log_priors[t] += 1
print(priors)
log_priors = [np.log(count/N) for count in log_priors]
print(log_priors)

[0.4, 0.6]
[-0.916290731874155, -0.5108256237659907]


### The Vocabulary

In [168]:
V = set([token for sent in sents for token in sent.split()])
n_V = len(V)

map_V = {word:pos for pos, word in enumerate(V)}

In [169]:
map_V

{'and': 0,
 'of': 1,
 'plain': 2,
 'laughs': 3,
 'no': 4,
 'summer': 5,
 'energy': 6,
 'just': 7,
 'boring': 8,
 'entirely': 9,
 'very': 10,
 'few': 11,
 'fun': 12,
 'surprises': 13,
 'file': 14,
 'the': 15,
 'lacks': 16,
 'most': 17,
 'predictable': 18,
 'powerful': 19}

### Convert the training documents into word counts

In [170]:
def doc_to_word_counts(doc):
    counts = [0]*n_V
    for token in doc.split():
        try:
            counts[map_V[token]] += 1
        except KeyError: # simply ignore words not in the vocabulary
            pass
    return counts

In [171]:
X = np.array([doc_to_word_counts(doc) for doc in docs])

$X$ now contains for each document (row) the counts per vocabulary word (column):

In [205]:
df_X = pd.DataFrame(X, columns=V)
df_X.index = docs
df_X

,and,of,plain,laughs,no,summer,energy,just,boring,entirely,very,few,fun,surprises,file,the,lacks,most,predictable,powerful
just plain boring,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0
entirely predictable and lacks energy,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,1,0
no surprises and very few laughs,1,0,0,1,1,0,0,0,0,0,1,1,0,1,0,0,0,0,0,0
very powerful,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1
the most fun file of the summer,0,1,0,0,0,1,0,0,0,0,0,0,1,0,1,2,0,1,0,0


### Training - Calculate the Likelihoods for the vocabulary words given class $c$
$$P(w_{i}|c)=\frac{count(w_i,c)+\alpha}{\sum_{w\in V}(count(w,c)+\alpha)}$$

First calculate the  $count(w_i,c)+\alpha$ for each class $c$ and vocabulary word $w_i$

In [234]:
counts_w_c = np.ones((K,n_V)) * alpha  # initialise with the +alpha counts for the Smoothing
for label,i in map_cls.items():  # loop over all labels and their positions in the cls array
    for doc in X[T == i]:  # select all docs of class c
        counts_w_c[i] += doc  # and add up the word counts

In [235]:
df_counts_w_c = pd.DataFrame(counts_w_c, columns=V)
df_counts_w_c.index = clss
df_counts_w_c

,and,of,plain,laughs,no,summer,energy,just,boring,entirely,very,few,fun,surprises,file,the,lacks,most,predictable,powerful
+,1.0,2.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,2.0,1.0,2.0,1.0,2.0,3.0,1.0,2.0,1.0,2.0
-,3.0,1.0,2.0,2.0,2.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,2.0,1.0,1.0,2.0,1.0,2.0,1.0


Calculate the denominator $\sum_{w\in V}(count(w,c)+\alpha)$

In [237]:
sums_per_c = np.sum(counts_w_c, axis=1)  # the alpha was already considered in the initialisation of counts_w_c! 
sums_per_c

array([29., 34.])

Calculate $\log P(w_{i}|c)$ per class

In [243]:
log_likelihood_w_c = np.log(counts_w_c / sums_per_c[:,None])
log_likelihood_w_c

array([[-3.36729583, -2.67414865, -3.36729583, -3.36729583, -3.36729583,
        -2.67414865, -3.36729583, -3.36729583, -3.36729583, -3.36729583,
        -2.67414865, -3.36729583, -2.67414865, -3.36729583, -2.67414865,
        -2.26868354, -3.36729583, -2.67414865, -3.36729583, -2.67414865],
       [-2.42774824, -3.52636052, -2.83321334, -2.83321334, -2.83321334,
        -3.52636052, -2.83321334, -2.83321334, -2.83321334, -2.83321334,
        -2.83321334, -2.83321334, -3.52636052, -2.83321334, -3.52636052,
        -3.52636052, -2.83321334, -3.52636052, -2.83321334, -3.52636052]])

### Inference
1. Convert document into word vector
2. Calculate posterior probabilities $\log P(c) + \sum_{t_i} \log P(w_i|c)$ for all classes $c$
3. Select the maximum probability and map to the correspoding class label

In [244]:
def infer(doc):
    # convert incoming document to word count vector
    x = np.array(doc_to_word_counts(doc))
    # calculate the posterior probabilities per class
    # elements of x are 0 if the corresponding vocabulary word does not appear in doc and 1 if it does
    # -> log_likelihood_w_c * x essentially selects the relevant log likelihoods
    class_probs = log_priors + np.sum(log_likelihood_w_c * x, axis=1)
    # select the class with the highest probability
    class_index = np.argmax(class_probs)
    # map class index back to label
    return clss[class_index]

In [251]:
infer('predictable with no fun')

'-'